In [1]:
!pip install langchain
!pip install langchain-community
!pip install gradio
!pip install pypdf
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguag

In [7]:
import os
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import InMemoryVectorStore
from langchain import LLMChain, PromptTemplate
from langchain.docstore.document import Document
from pypdf import PdfReader
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from google.colab import userdata

In [8]:
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')

In [9]:
loader=PyPDFLoader("/content/file1.pdf")
pages=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
texts=text_splitter.split_documents(pages)

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GEMINI_API_KEY)

In [11]:
docsearch = InMemoryVectorStore.from_documents(texts,embeddings)
docsearch.embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7ab5888af710>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7ab583b09110>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [20]:
template = """You are a teaching expert:

{context}

Now, based on this information, answer the following question:

{question}

Provide a concise and accurate response below:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"], template=template
)

retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [19]:
llm=ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY,model="gemini-2.5-flash")
result=llm.invoke("What are the usecases of LLMs?")
print(result)

content='Large Language Models (LLMs) are incredibly versatile and have a wide range of use cases across various industries and personal applications. Their core strength lies in understanding, generating, and manipulating human language.\n\nHere\'s a breakdown of key use cases:\n\n### I. Content Generation & Creation\n\n1.  **Text Generation:**\n    *   **Articles & Blogs:** Drafting outlines, full articles, or specific sections on various topics.\n    *   **Marketing Copy:** Generating ad copy, social media posts, email newsletters, product descriptions, website content.\n    *   **Creative Writing:** Assisting with story plots, character descriptions, poetry, song lyrics, and script outlines.\n    *   **Summarization:** Condensing long documents, articles, meeting transcripts, or reports into concise summaries.\n    *   **Email & Communication:** Drafting professional emails, responses, meeting agendas, and internal communications.\n    *   **Reports & Proposals:** Generating sectio

In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}

)

In [18]:
query = "bear"
result = qa_chain.invoke(query)
print(result['result'])

The numbers 5186 and 5178 represent the **number of citations** for the European Conference on Computer Vision (ECCV) and The Thirty-Seventh AAAI Conference on Artificial Intelligence (AAAI-23), respectively.


In [22]:
def get_text_response(message, history):
  query = message
  result = qa_chain.invoke(query)
  response= result['result']
  return response

demo = gr.ChatInterface(fn= get_text_response, examples=["How are you doing?","What are your interests?","Which places do you like to visit?"],title="RAG")
if __name__ == "__main__":
    demo.launch() #To create a public link, set `share=True` in `launch()`. To enable errors and logs, set `debug=True` in `launch()`

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://233007e07f9ad3d21e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
